In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
from sklearn.preprocessing import OneHotEncoder

warnings.filterwarnings("ignore")

data = pd.read_csv("M:/DataSet/usaccident/real/SC.csv",index_col=0)

In [2]:
def missing_values_table(df):
        #计算总的缺失值
        mis_val = df.isnull().sum()
        
        #计算缺失值的百分比
        mis_val_percent = 100 * df.isnull().sum() / len(df)
   
        #把结果制成表格
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        #对列重命名，第一列：Missing Values，第二列：% of Total Values
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        #根据百分比对表格进行降序排列
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        #打印总结信息：总的列数，有数据缺失的列数
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
       
        # 返回带有缺失值信息的dataframe
        return mis_val_table_ren_columns

1.数据基本信息
=

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212712 entries, 146278 to 4232214
Data columns (total 49 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     212712 non-null  object 
 1   Source                 212712 non-null  object 
 2   TMC                    184622 non-null  float64
 3   Severity               212712 non-null  int64  
 4   Start_Time             212712 non-null  object 
 5   End_Time               212712 non-null  object 
 6   Start_Lat              212712 non-null  float64
 7   Start_Lng              212712 non-null  float64
 8   End_Lat                28090 non-null   float64
 9   End_Lng                28090 non-null   float64
 10  Distance(mi)           212712 non-null  float64
 11  Description            212712 non-null  object 
 12  Number                 132097 non-null  float64
 13  Street                 212712 non-null  object 
 14  Side                   212712 

In [4]:
missing_values_table(data)

Your selected dataframe has 49 columns.
There are 17 columns that have missing values.


,Missing Values,% of Total Values
End_Lat,184622,86.8
End_Lng,184622,86.8
Precipitation(in),93985,44.2
Wind_Chill(F),91101,42.8
Number,80615,37.9
TMC,28090,13.2
Wind_Speed(mph),26870,12.6
Visibility(mi),4269,2.0
Weather_Condition,4185,2.0
Humidity(%),3975,1.9


In [5]:
data.nunique()

ID                       212712
Source                        3
TMC                          16
Severity                      4
Start_Time               205855
End_Time                 204398
Start_Lat                 77905
Start_Lng                 77607
End_Lat                   15848
End_Lng                   15890
Distance(mi)               3100
Description              103506
Number                     9140
Street                    23021
Side                          2
City                        325
County                       47
State                         1
Zipcode                   47261
Country                       1
Timezone                      1
Airport_Code                 57
Weather_Timestamp         97361
Temperature(F)              488
Wind_Chill(F)               397
Humidity(%)                  94
Pressure(in)                257
Visibility(mi)               30
Wind_Direction               24
Wind_Speed(mph)              63
Precipitation(in)           122
Weather_

In [6]:
dropCols = ["End_Lat","End_Lng",
            "Timezone","Country","State","Turning_Loop",'Weather_Timestamp']
data.drop(dropCols,axis=1,inplace=True)
data.shape

(212712, 42)

数据预处理
=

1.时间特征

In [7]:
data['Start_Time'] = pd.to_datetime(data['Start_Time'])
data['End_Time'] = pd.to_datetime(data['End_Time'])

data['Year'] = data['Start_Time'].dt.year
data['Month'] = data['Start_Time'].dt.month
data['Day'] = data['Start_Time'].dt.day
data['Hour'] = data['Start_Time'].dt.hour
data['Weekday'] = data['Start_Time'].dt.weekday
data['Duration'] = (data['End_Time'] - data['Start_Time'])/np.timedelta64(1,'m')
data['Duration'] = data['Duration'].astype(int)

data.drop(['Start_Time','End_Time'],axis=1,inplace=True)

data[['Year','Month','Day','Hour','Weekday','Duration']].head(5)

,Year,Month,Day,Hour,Weekday,Duration
146278,2016,11,30,16,2,44
146285,2016,11,30,17,2,44
146291,2016,11,30,18,2,44
146294,2016,11,30,19,2,44
146295,2016,11,30,19,2,44


2.缺失值

In [8]:
''' TMC:用众数填充 '''

data['TMC'].value_counts(ascending=False)

201.0    176866
241.0      6514
245.0       496
202.0       212
229.0       129
343.0       112
222.0        72
247.0        53
406.0        49
236.0        41
244.0        34
206.0        25
203.0        10
246.0         5
248.0         3
339.0         1
Name: TMC, dtype: int64

In [9]:
data['TMC'] = data['TMC'].fillna(201)
data['TMC'].isnull().sum()

0

In [10]:
''' Wind_Speed : calm风向填充为0，其余按地区时间填充'''

data['Wind_Direction'].value_counts(ascending=False)

CALM        28511
Calm        22086
SW          14428
WSW         13192
NE          13006
NNE         11986
SSW         11322
ENE          9517
W            6528
West         6345
North        6329
N            6113
VAR          5982
South        5963
S            5819
SSE          5677
WNW          5333
NNW          4808
Variable     4755
E            4518
NW           4329
SE           4158
ESE          4150
East         4041
Name: Wind_Direction, dtype: int64

In [11]:
data.loc[data['Wind_Direction']=='Calm','Wind_Speed(mph)'] = 0
data['Wind_Speed(mph)'] = data.groupby(['Street','Year','Month','Day','Hour'])['Wind_Speed(mph)'].apply(lambda x: x.fillna(x.mean()))
data['Wind_Speed(mph)'] = data['Wind_Speed(mph)'].fillna(data['Wind_Speed(mph)'].mean())
data['Wind_Speed(mph)'].isnull().sum()

0

In [12]:
''' Temperature:用城市各年各月的平均值填充 '''

data['Temperature(F)'] = data.groupby(['Street','Year','Month','Day','Hour'])['Temperature(F)'].apply(lambda x: x.fillna(x.mean()))
data['Temperature(F)'] = data['Temperature(F)'].fillna(data['Temperature(F)'].mean())
data['Temperature(F)'].isnull().sum()

0

In [13]:
''' Humidity:同Temperature '''

data['Humidity(%)'] = data.groupby(['Street','Year','Month','Day','Hour'])['Humidity(%)'].apply(lambda x: x.fillna(x.mean()))
data['Humidity(%)'] = data['Humidity(%)'].fillna(data['Humidity(%)'].mean())
data['Humidity(%)'].isnull().sum()

0

In [14]:
''' Visibility:同Temperature '''

data['Visibility(mi)'] = data.groupby(['Street','Year','Month','Day','Hour'])['Visibility(mi)'].apply(lambda x: x.fillna(x.mean()))
data['Visibility(mi)'] = data['Visibility(mi)'].fillna(data['Visibility(mi)'].mean())
data['Visibility(mi)'].isnull().sum()

0

In [15]:
''' Precipitation(in):同Temperature '''

data['Precipitation(in)'] = data.groupby(['Street','Year','Month','Day','Hour'])['Precipitation(in)'].apply(lambda x: x.fillna(x.mean()))
data['Precipitation(in)'] = data['Precipitation(in)'].fillna(data['Precipitation(in)'].mean())
data['Precipitation(in)'].isnull().sum()

0

In [16]:
''' Wind_Chill(F):同Temperature '''

data['Wind_Chill(F)'] = data.groupby(['Street','Year','Month','Day','Hour'])['Wind_Chill(F)'].apply(lambda x: x.fillna(x.mean()))
data['Wind_Chill(F)'] = data['Wind_Chill(F)'].fillna(data['Wind_Chill(F)'].mean())
data['Wind_Chill(F)'].isnull().sum()

0

In [17]:
''' Pressure(in):同Temperature '''

data['Pressure(in)'] = data.groupby(['Street','Year','Month','Day','Hour'])['Pressure(in)'].apply(lambda x: x.fillna(x.mean()))
data['Pressure(in)'] = data['Pressure(in)'].fillna(data['Pressure(in)'].mean())
data['Pressure(in)'].isnull().sum()

0

In [18]:
data.drop(['Number'],axis=1,inplace=True)
missing_values_table(data)

Your selected dataframe has 45 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
Weather_Condition,4185,2.0
Wind_Direction,3816,1.8
Airport_Code,1926,0.9
Zipcode,2,0.0


In [19]:
data[['Airport_Code','Zipcode']].head(5)

,Airport_Code,Zipcode
146278,KAND,29655-9004
146285,KGRD,29646-3029
146291,KEOE,29108
146294,KGRD,29649-1300
146295,KAGS,29842-7685


In [23]:
data['Weather_Condition'].value_counts()[:5]

Fair             57502
Clear            44275
Mostly Cloudy    21723
Cloudy           17348
Partly Cloudy    15853
Name: Weather_Condition, dtype: int64

In [31]:
''' Weather_Condition:用众数 '''

data['Weather_Condition'] = data.groupby(['Street','Year','Month','Day'])['Weather_Condition'].apply(lambda x: x.fillna(x.mode()))
data['Weather_Condition'] = data['Weather_Condition'].fillna(data['Weather_Condition'].mode()[0])
data['Weather_Condition'].isnull().sum()

0

In [32]:
''' Wind_Direction:用众数 '''

data['Wind_Direction'] = data.groupby(['Street','Year','Month','Day'])['Wind_Direction'].apply(lambda x: x.fillna(x.mode()))
data['Wind_Direction'] = data['Wind_Direction'].fillna(data['Wind_Direction'].mode()[0])
data['Wind_Direction'].isnull().sum()

0

In [33]:
missing_values_table(data)

Your selected dataframe has 45 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
Airport_Code,1926,0.9
Zipcode,2,0.0


In [34]:
''' Airport_Code:用众数 '''

data['Airport_Code'] = data.groupby(['Street'])['Airport_Code'].apply(lambda x: x.fillna(x.mode()))
data['Airport_Code'] = data['Airport_Code'].fillna(data['Airport_Code'].mode()[0])
data['Airport_Code'].isnull().sum()

0

In [35]:
''' Zip_Code:用众数 '''

data['Zipcode'] = data.groupby(['Street'])['Zipcode'].apply(lambda x: x.fillna(x.mode()))
data['Zipcode'] = data['Zipcode'].fillna(data['Zipcode'].mode()[0])
data['Zipcode'].isnull().sum()

0

In [36]:
data.shape

(212712, 45)

In [37]:
data.to_csv("M:/DataSet/usaccident/real/fillna.csv")